In [ ]:
import numpy as np
import random
import copy
import jax.numpy as jnp
import jax

In [2]:
class Envrionment:
    
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    

In [ ]:
# class TabularMDP(Evironment):
    
#     def __init__(self,n_state: int,n_action: int,ep_len: int,gamma: float):
#         '''
#         n_state -> number of states
#         n_action -> number of actions
#         ep_len -> episode_length
        
#         '''
#         self.n_state = n_state
#         self.n_action = n_action
#         self.ep_len = ep_len
#         self.timestep = 0
#         self.gamma = gamma
        
#         #報酬と遷移確率の定義: [0,1]
#         reward = np.random.rand(n_state,n_action)
#         self.reward = reward / np.abs(reward).max
#         P = np.random.rand(n_state,n_action,n_state)
#         self.P = P/ np.sum(P,axis=-1,keepdims=True)
        
#         #訪問回数を定義
#         self.N_sa = np.zeros(self.n_state,self.n_action)
#         self.N_sa_s = np.zeros(self.n_state,self.n_action,self.n_state)
#         self.
        
#     def reset(self):
#         self.timestep = 0
#         self.N_sa = np.zeros(self.n_state,self.n_action)
#         self.N_sa_s = np.zeros(self.n_state,self.n_action,self.n_state)
        
        
        
        

In [5]:
!pip install gym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 36.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827634 sha256=771779f2fab70b6ab8f6eb9edd1305643988aef7ee21fa8ebcd5b9712f554d24
  Stored in directory: /home/jupyter/.cache/pip/wheels/4d/c8/dc/d08577bffa680f083f04448e81e4f176d0e45a13eb16f93c0c
Successfully built gym

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import gym

In [42]:
env = gym.make('FrozenLake-v1', desc=None, map_name="8x8", is_slippery=False)

In [43]:
state = env.reset()

In [47]:
state

(0, {'prob': 1})

In [44]:
env.observation_space.n

64

In [45]:
env.step(2)

(1, 0.0, False, False, {'prob': 1.0})

In [35]:
class UCBVI:
    def __init__(self,env,K: int,H: int):
        self.env = env
        self.A = env.action_space.n
        self.S = env.observation_space.n
        self.N_sa_s = jnp.zeros([self.S,self.A,self.S])
        self.N_sa = jnp.zeros([self.S,self.A])
        self.P = jnp.zeros([self.S,self.A,self.S])
        self.Q = jnp.zeros([H,self.S,self.A])
        self.delta = 1/3
        self.K = K
        self.H = H
        self.N = jnp.zeros([H,self.S,self.A])
    
    
    
    def choice_a(self,h,s):
        return np.argmax(self.Q[h][s],axis = -1)
        
    def update_transit(self,s,a,n_s):
        self.P[s,a,n_s] = self.N_sa_s[s,a,n_s]/self.N_sa[s,a]
    
    def update_count(self,s,a,n_s,h):
        self.N[h,s,a] +=1
        self.N_sa_s[s,a,n_s] += 1
        self.N_sa[s,a] += 1
        
    def update_Q(self,s,a,h,n_s,reward):
        V_next = self.Q[h,n_s,:].max(axis = -1)
        # b = bonus(self.P[s,a,n_s],V_next,self.N_sa_s[s,a,n_s],self.N[h,s,a])
        b = self.bonus_1(s,a)
        
        self.Q = min(self.Q[h,s,a],self.H,reward + self.P[s,a,n_s] * self.V_next + b)
        
    
    def learn(s,a,n_s,h,reward):
        self.update_count(s,a,n_s,h)
        self.update_transit(s,a,n_s)
        self.update_Q(s,a,h,n_s,reward)
        
    
    def bonus_1(self,s,a):
        L = np.log(5*self.S*self.A*self.K*self.H/self.delta)
        return 7 * self.H * L * np.sqrt(1/self.N_sa[s,a])
    
        
        
        
        
    
        
    
        
    

In [36]:
K = 10000
agent = UCBVI(env,K = K,H = 200)

In [39]:
from tqdm import tqdm

In [58]:
for k in tqdm(range(K)):
    env.reset()
    done = 0
    for h in range(200):
        
        s = env.reset()[0]
        
        a = agent.choice_a(h,s)
        print(a)
       
        if k == K-1:
            print(a)
        n_s, reward, done, _,_ = env.step(a)
        
        if done != 1:
            agent.learn(s,a,n_s,h,reward)
        else:
            break
    

  0%|          | 0/10000 [00:00<?, ?it/s]

0


TypeError: unhashable type: 'DeviceArray'